In [ ]:
import numpy as np
import pandas as pd
import sys, os
import csv
from pathlib import Path
from natsort import natsorted
sys.path.append('/scicore/home/nimwegen/degroo0000/Bonsai-data-representation')
from scipy.spatial import distance
import matplotlib.pyplot as plt
from paper_figure_scripts_and_notebooks.simulating_datasets.analyzing_simulated_datasets.knn_recall_helpers import get_pdists_on_tree, Dataset, compare_pdists_to_truth_per_cell, do_pca, fit_umap, fit_phate
from scipy.spatial.distance import squareform
import logging
logging.getLogger("umap").disabled = True
log_level = logging.ERROR
FORMAT = '%(asctime)s %(name)s %(funcName)s %(message)s'
logging.basicConfig(format=FORMAT, datefmt='%H:%M:%S',
                    level=log_level)
plt.set_loglevel(level='warning')

In [ ]:
"""--------------------Layout settings----------------------"""
SMALL_SIZE = 9
MEDIUM_SIZE = 11
BIGGER_SIZE = 13

plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### Get list of datasets and prepare dictionaries to hold data

In [ ]:
base_folder = '/scicore/home/nimwegen/degroo0000/Bonsai-data-representation/slurm_runs_pipeline/output/no_noise_simulated'
poss_datasets = os.listdir(base_folder)
all_datasets = []
for dataset in poss_datasets:
    if os.path.exists(os.path.join(base_folder,dataset,'bonsai')):
        bonsai_folders = os.listdir(os.path.join(base_folder,dataset,'bonsai'))
        if any(s.startswith("final_bonsai") for s in bonsai_folders):
            all_datasets.append(dataset)
n_datasets = len(all_datasets)
print("We've got {} datasets.".format(n_datasets))

In [ ]:
type_suffix = 'pcatransformed'

In [ ]:
cell_numbers = [7509]
gene_numbers = [10, 50, 100] #, 500, 1000]

In [ ]:
datasets = []
for ind_dataset, dataset in enumerate(all_datasets):
    split_dataset = dataset.split('_')
    if split_dataset[-1] != type_suffix:
        continue
    else:
        nCell = split_dataset[-2]
        nGene = split_dataset[-3]
    n_cell = int(nCell[5:])
    n_gene = int(nGene[5:])
    if (n_cell not in cell_numbers) or (n_gene not in gene_numbers):
        continue
    print(nCell, nGene)
    datasets.append(dataset)
datasets = natsorted(datasets)
n_datasets = len(datasets)
print("We've got {} datasets.".format(n_datasets))

In [ ]:
if n_datasets > 10:
    tab20 = plt.get_cmap("tab20")
else:
    tab20 = plt.get_cmap("tab10")
colors = [tab20(i) for i in range(n_datasets)]

In [ ]:
info_dss = {}
for ind_dataset, dataset in enumerate(datasets):
    split_dataset = dataset.split('_')
    if split_dataset[-1] != type_suffix:
        continue
    else:
        nCell = split_dataset[-2]
        nGene = split_dataset[-3]
    print(nCell, nGene)
    n_cell = int(nCell[5:])
    n_gene = int(nGene[5:])
    if (n_cell not in cell_numbers) or (n_gene not in gene_numbers):
        continue
    dataset_dict = {'n_cell': n_cell, 'n_gene': n_gene, 
                    'plt_coords': (gene_numbers.index(n_gene), cell_numbers.index(n_cell)), 
                    'color': colors[ind_dataset], 'index': ind_dataset, 'dataset_path': os.path.join(base_folder,dataset)}
    info_dss[dataset] = dataset_dict

In [ ]:
pdists_true_dss = {dataset: None for dataset in datasets}
pdists_inferred_dss = {dataset: None for dataset in datasets}

### Create function to get a many-time use fig, ax combination

In [ ]:
def get_fig_ax(xlabel=None, ylabel=None, title=None, sharex=None, sharey=None):
    # ncols = np.minimum(int(np.ceil(np.sqrt(n_datasets))),4)
    # nrows = int(np.ceil(n_datasets/ncols))
    # fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4, nrows*3))
    fig, axs = plt.subplots(ncols=len(cell_numbers), nrows=len(gene_numbers), 
                            figsize=(len(cell_numbers)*4, len(gene_numbers)*3), 
                            sharex=sharex, sharey=sharey)
    axs_flat = axs.flatten()
    for ax in axs.flat:
        ax.tick_params(axis='x', which='both', labelbottom=True)
        ax.tick_params(axis='y', which='both', labelleft=True)
    
    for ind_ds, dataset in enumerate(datasets):
        info = info_dss[dataset]
        ax_row, ax_col = info['plt_coords']
        if len(cell_numbers) == 1:
            ax = axs[ax_row]
        else:
            ax = axs[ax_row, ax_col]
        ax.set_title('|C| = {}, |G| = {}'.format(info['n_cell'], info['n_gene']))

    for ax_row in range(len(gene_numbers)):
        for ax_col in range(len(cell_numbers)):
            if len(cell_numbers) == 1:
                ax = axs[ax_row]
            else:
                ax = axs[ax_row, ax_col]
            if (xlabel is not None) and (ax_row == (len(gene_numbers)-1)):
                ax.set_xlabel(xlabel)
            if (ylabel is not None)  and (ax_col == 0):
                ax.set_ylabel(ylabel)
    if title is not None:
        fig.suptitle(title)
    
    return fig, axs

### First get pairwise distances of ground truth data

In [ ]:
output_folder_dss = {}
for dataset in datasets:
    sanity_folder = os.path.join(base_folder, dataset, 'Sanity')
    true_deltas = pd.read_csv(os.path.join(sanity_folder, 'delta_vmax.txt'), sep='\t', header=None).values.astype(dtype=float)
    output_folder = os.path.join(base_folder, dataset)
    output_folder_dss[dataset] = output_folder
    Path(os.path.join(output_folder, 'intermediate_files')).mkdir(parents=True, exist_ok=True)
    num_dims = true_deltas.shape[0]
    true_dists = distance.pdist(true_deltas.T, metric='sqeuclidean') / num_dims
    pdists_true_dss[dataset] = true_dists
    np.save(os.path.join(output_folder, 'intermediate_files', 'delta_true_pdists.npy'), true_dists,
                allow_pickle=False)

In [ ]:
fig, axs = get_fig_ax(xlabel='Squared Eucl. dist.', ylabel='Number of cell-cell pairs', title='True pairwise distances', sharex=True, sharey='col')

# Visualize the true distances
for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    ax_row, ax_col = info['plt_coords']
    print(dataset)
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax.hist(pdists_true_dss[dataset], bins=100, color=info['color'], log=True)
plt.tight_layout()

### Then get inferred pairwise distances by getting summed branch lengths

In [ ]:
# I first need the cell IDs, and paths to the newick files
cell_ids_dss = {}
for dataset in datasets:
    info = info_dss[dataset]
    cell_id_path = os.path.join(info['dataset_path'], 'Sanity', 'cellID.txt')
    cell_ids = []
    with open(cell_id_path, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            cell_ids.append(row[0])
    cell_ids_dss[dataset] = cell_ids

    for bonsai_folder in os.listdir(os.path.join(info['dataset_path'],'bonsai')):
        if bonsai_folder.startswith('final_bonsai'):
            info['bonsai_path'] = os.path.join(info['dataset_path'],'bonsai', bonsai_folder)

In [ ]:
bonsai_dists_dss = {}
for ind_dataset, dataset in enumerate(datasets):
    output_folder = output_folder_dss[dataset]
    info = info_dss[dataset]
    print(dataset)
    nwk_path = os.path.join(info['bonsai_path'], 'tree.nwk')
    bonsai_dists = get_pdists_on_tree(nwk_path, cell_ids_dss[dataset])
    bonsai_dists_dss[dataset] = bonsai_dists
    np.save(os.path.join(output_folder, 'intermediate_files', 'bonsai_pdists.npy'), bonsai_dists,
                allow_pickle=False)

In [ ]:
fig, axs = get_fig_ax(xlabel='Bonsai dist.', ylabel='Number of cell-cell pairs', title='Bonsai distances', sharex=True, sharey='col')

# Visualize the true distances
for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax_row, ax_col = info['plt_coords']
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax.hist(bonsai_dists_dss[dataset], bins=100, color=info['color'], log=True)
plt.tight_layout()

### Create comparison plot, true vs pairwise distance

In [ ]:
bonsai_objcts_dss = {}
for dataset in datasets:
    print("\n\nTreating dataset {}\n".format(dataset))
    info = info_dss[dataset]
    ax_row, ax_col = info['plt_coords']
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    true_dists = pdists_true_dss[dataset]
    bonsai_dists = bonsai_dists_dss[dataset]

    true_objct = Dataset(distances=true_dists, data_type='delta_true', data_id='delta_true')
    data_id = dataset
    bonsai_objct = Dataset(distances=bonsai_dists, data_type='bonsai', data_id=data_id)
    fig_fake, ax_fake = plt.subplots()
    avg_rel_diffs, R_vals = compare_pdists_to_truth_per_cell([true_objct, bonsai_objct], 
                                                             make_fig=True, axs=ax, 
                                                             set_lims=False, return_Rvals=True, 
                                                             loglog_corr=False)
    bonsai_objcts_dss[dataset] = bonsai_objct

## Get one figure where total correlation and per-cell correlation are put together

In [ ]:
ncols = 2
fig, axs = plt.subplots(ncols=ncols, nrows=len(gene_numbers), 
                        figsize=(ncols*4, len(gene_numbers)*3))
for i in range(1, len(gene_numbers)):
    axs[i, 1].sharey(axs[0, 1])
    axs[i, 1].sharex(axs[0, 1])
axs_flat = axs.flatten()
for ax in axs.flat:
    ax.tick_params(axis='x', which='both', labelbottom=True)
    ax.tick_params(axis='y', which='both', labelleft=True)

for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    ax_row, ax_col = info['plt_coords']
    ax = axs[ax_row, 0]
    ax.set_title('|C| = {}, |G| = {}'.format(info['n_cell'], info['n_gene']))
    ax = axs[ax_row, 1]
    ax.set_title('|C| = {}, |G| = {}'.format(info['n_cell'], info['n_gene']))

xlabels = ['True sq. Eucl. distance', 'Per-cell correlations (Pearson R^2)']
ylabels = ['Bonsai summed branch lengths', 'Number of cells']
for ax_row in range(len(gene_numbers)):
    for ax_col in range(2):
        ax = axs[ax_row, ax_col]
        if (xlabels is not None) and (ax_row == (len(gene_numbers)-1)):
            ax.set_xlabel(xlabels[ax_col])
        if (ylabels is not None):
            ax.set_ylabel(ylabels[ax_col])

loglog_corr = False

# Visualize the true distances
for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    ax_row, ax_col = info['plt_coords']
    ax = axs[ax_row, ax_col]
    true_dists = pdists_true_dss[dataset]
    bonsai_dists = bonsai_dists_dss[dataset]
    ax.scatter(true_dists, bonsai_dists, s=2, alpha=0.5, color=info['color'])

    if loglog_corr:
        ax.set_xscale('log')
        ax.set_yscale('log')
        
    nonzeros_true = true_dists != 0
    nonzeros_data = bonsai_dists != 0
    nonzeros_all = nonzeros_data * nonzeros_true

    # Add correlation line
    if loglog_corr:
        log_truedists = true_dists.copy()
        log_truedists[nonzeros_true] = np.log(true_dists[nonzeros_true])
    else:
        log_truedists = true_dists

    if loglog_corr:
        log_truedists_nonzeros_cell = log_truedists[nonzeros_all]
        log_datadists_cell = np.log(bonsai_dists[nonzeros_all])
    else:
        log_truedists_nonzeros_cell = log_truedists
        log_datadists_cell = bonsai_dists

    Clog = np.cov(np.vstack((log_truedists_nonzeros_cell - np.mean(log_truedists_nonzeros_cell),
                             log_datadists_cell - np.mean(log_datadists_cell))))
    pearsonR = Clog[0, 1] / np.sqrt(Clog[0, 0] * Clog[1, 1])

    eigVals, eigVecs = np.linalg.eig(Clog)
    max_eigval = np.argmax(eigVals)
    slopepca1log = eigVecs[1, max_eigval] / eigVecs[0, max_eigval]
    regLineXlog = np.linspace(log_truedists_nonzeros_cell.min(), log_truedists_nonzeros_cell.max(), 20)
    regLineYlog = slopepca1log * (
            regLineXlog - log_truedists_nonzeros_cell.mean()) + log_datadists_cell.mean()

    if loglog_corr:
        # ax.plot(np.exp(regLineXlog), np.exp(regLineYlog), '--', lw=2, c='black', zorder=10)
        ax.plot(np.exp(regLineXlog), np.exp(regLineXlog), '--', lw=2, c='black', zorder=10)
    else:
        # ax.plot(regLineXlog, regLineYlog, '--', lw=2, c='black', zorder=10)
        ax.plot(regLineXlog, regLineXlog, '--', lw=2, c='black', zorder=10)
    corr_measure = 'Rsq'
    # corr_measure = '-log(1-R^2)'
    if corr_measure == 'Rsq':
        corr_label = pearsonR ** 2
    elif corr_measure == '-log(1-R^2)':
        corr_label = -np.log(1-pearsonR ** 2)
    # ax.text(0.98, 0.01,
    #              "Slope: {:.2f},\n{}={:.2f}".format(slopepca1log, corr_measure, corr_label),
    #              horizontalalignment='right', verticalalignment='bottom',
    #              transform=ax.transAxes, fontsize=12)
    ax.text(0.98, 0.01,
             "{}={:.2f}".format(corr_measure, corr_label),
             horizontalalignment='right', verticalalignment='bottom',
             transform=ax.transAxes, fontsize=12)


for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    ax_row, _ = info['plt_coords']
    ax = axs[ax_row, 1]
    bonsai_objct = bonsai_objcts_dss[dataset]
    pearsonRs = bonsai_objct.pearsonRs
    ax.hist(pearsonRs ** 2, bins=100, density=False, color=info['color'], log=False)

plt.tight_layout()

In [ ]:
# fig.savefig(os.path.join(os.getcwd(), 'nonoise_dataset_accuracy_of_distance_estimates.png'), dpi=300)
# fig.savefig(os.path.join(os.getcwd(), 'nonoise_dataset_accuracy_of_distance_estimates.svg'))

In [ ]:
print("Stored figures at: {}".format(os.path.join(os.getcwd(), 'nonoise_dataset_accuracy_of_distance_estimates.png')))

In [ ]:
ncols = len(gene_numbers)
fig, axs = plt.subplots(nrows=1, ncols=len(gene_numbers), 
                        figsize=(ncols*3, 3))
# axs_flat = axs.flatten()
# for ax in axs.flat:
#     ax.tick_params(axis='x', which='both', labelbottom=True)
#     ax.tick_params(axis='y', which='both', labelleft=True)

for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    ax_row, ax_col = info['plt_coords']
    ax = axs[ax_row]
    ax.set_title('Number of dimensions: {}'.format(info['n_gene']))
    # ax = axs[ax_row]
    # ax.set_title('|C| = {}, |G| = {}'.format(info['n_cell'], info['n_gene']))

xlabels = ['True squared\nEuclidean distance']  #, 'Per-cell correlations (Pearson R^2)']
ylabels = ['Summed Bonsai\nbranch lengths'] # , 'Number of cells']
for ax_col in range(len(gene_numbers)):
    ax = axs[ax_col]
    if (xlabels is not None):
        ax.set_xlabel(xlabels[0])
    if (ylabels is not None) and (ax_col == 0):
            ax.set_ylabel(ylabels[0])

loglog_corr = False

# Visualize the true distances
for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    ax_row, ax_col = info['plt_coords']
    ax = axs[ax_row]
    true_dists = pdists_true_dss[dataset]
    bonsai_dists = bonsai_dists_dss[dataset]
    ax.scatter(true_dists, bonsai_dists, s=2, alpha=0.5, color=info['color'])

    if loglog_corr:
        ax.set_xscale('log')
        ax.set_yscale('log')
        
    nonzeros_true = true_dists != 0
    nonzeros_data = bonsai_dists != 0
    nonzeros_all = nonzeros_data * nonzeros_true

    # Add correlation line
    if loglog_corr:
        log_truedists = true_dists.copy()
        log_truedists[nonzeros_true] = np.log(true_dists[nonzeros_true])
    else:
        log_truedists = true_dists

    if loglog_corr:
        log_truedists_nonzeros_cell = log_truedists[nonzeros_all]
        log_datadists_cell = np.log(bonsai_dists[nonzeros_all])
    else:
        log_truedists_nonzeros_cell = log_truedists
        log_datadists_cell = bonsai_dists

    Clog = np.cov(np.vstack((log_truedists_nonzeros_cell - np.mean(log_truedists_nonzeros_cell),
                             log_datadists_cell - np.mean(log_datadists_cell))))
    pearsonR = Clog[0, 1] / np.sqrt(Clog[0, 0] * Clog[1, 1])

    eigVals, eigVecs = np.linalg.eig(Clog)
    max_eigval = np.argmax(eigVals)
    slopepca1log = eigVecs[1, max_eigval] / eigVecs[0, max_eigval]
    regLineXlog = np.linspace(log_truedists_nonzeros_cell.min(), log_truedists_nonzeros_cell.max(), 20)
    regLineYlog = slopepca1log * (
            regLineXlog - log_truedists_nonzeros_cell.mean()) + log_datadists_cell.mean()

    if loglog_corr:
        # ax.plot(np.exp(regLineXlog), np.exp(regLineYlog), '--', lw=2, c='black', zorder=10)
        ax.plot(np.exp(regLineXlog), np.exp(regLineXlog), '--', lw=2, c='black', zorder=10)
    else:
        # ax.plot(regLineXlog, regLineYlog, '--', lw=2, c='black', zorder=10)
        ax.plot(regLineXlog, regLineXlog, '--', lw=2, c='black', zorder=10)
    corr_measure = 'Rsq'
    # corr_measure = '-log(1-R^2)'
    if corr_measure == 'Rsq':
        corr_label = pearsonR ** 2
    elif corr_measure == '-log(1-R^2)':
        corr_label = -np.log(1-pearsonR ** 2)
    # ax.text(0.98, 0.01,
    #              "Slope: {:.2f},\n{}={:.2f}".format(slopepca1log, corr_measure, corr_label),
    #              horizontalalignment='right', verticalalignment='bottom',
    #              transform=ax.transAxes, fontsize=12)
    ax.text(0.98, 0.01,
             "{}={:.2f}".format(corr_measure, corr_label),
             horizontalalignment='right', verticalalignment='bottom',
             transform=ax.transAxes, fontsize=12)

plt.tight_layout()

In [ ]:
fig.savefig(os.path.join(os.getcwd(), 'one_row_nonoise_dataset_accuracy_of_distance_estimates.png'), dpi=600)
# fig.savefig(os.path.join(os.getcwd(), 'one_row_nonoise_dataset_accuracy_of_distance_estimates.svg'))
print("Stored figures at: {}".format(os.path.join(os.getcwd(), 'one_row_nonoise_dataset_accuracy_of_distance_estimates.png')))

# Add comparison to UMAP, PCA and PHATE

In [ ]:
pca_projected_dss = {}
PCA_COMPS = [2, 10]
for ind_ds, dataset in enumerate(datasets):
    sanity_folder = os.path.join(base_folder, dataset, 'Sanity')
    true_deltas = pd.read_csv(os.path.join(sanity_folder, 'delta_vmax.txt'), sep='\t', header=None).values.astype(dtype=float)
    num_dims = true_deltas.shape[0]  
    info = info_dss[dataset]
    pca_projected = do_pca(true_deltas, n_comps_list=PCA_COMPS)
    pca_projected_dss[dataset] = pca_projected
    # for n_comps, pca_proj in pca_projected.items():
    #     np.save('tmp_pca_{}.npy'.format(n_comps), pca_proj, allow_pickle=False)

In [ ]:
umap_projected_dss = {}
PCA_COMPS = [2, 10]
for ind_ds, dataset in enumerate(datasets):
    pca_projected = pca_projected_dss[dataset]
    umap_projected = {}
    for n_comps, pca_proj in pca_projected.items():
        if n_comps == 2:
            continue
        umap_projected[n_comps] = fit_umap(pca_proj, random_state=None, n_neighbors=15, min_dist=0.1,
                                           n_components=2,
                                           metric='euclidean',
                                           make_plot=False, title='')
    umap_projected_dss[dataset] = umap_projected

In [ ]:
phate_projected_dss = {}
for ind_ds, dataset in enumerate(datasets):
    phate_projected = {}
    phate_projected['all'] = fit_phate(true_deltas)
    phate_projected_dss[dataset] = phate_projected

In [ ]:
pca_dists_dss = {}
for ind_ds, dataset in enumerate(datasets):
    output_folder = output_folder_dss[dataset]
    pca_projected = pca_projected_dss[dataset]
    for n_comps, pca_proj in pca_projected.items():
        if n_comps != 2:
            continue
        pca_dists = distance.pdist(pca_proj.T, metric='sqeuclidean') / 2
    pca_dists_dss[dataset] = pca_dists
    np.save(os.path.join(output_folder, 'intermediate_files', 'pca_{}_pdists.npy'.format(n_comps)),
                        pca_dists, allow_pickle=False)

In [ ]:
umap_dists_dss = {}
for ind_ds, dataset in enumerate(datasets):
    output_folder = output_folder_dss[dataset]
    umap_projected = umap_projected_dss[dataset]
    for n_comps, umap_proj in umap_projected.items():
        if n_comps == 2:
            continue
        umap_dists = distance.pdist(umap_proj.T, metric='sqeuclidean') / 2
        np.save('tmp_umap_{}_pdists.npy'.format(n_comps), umap_dists, allow_pickle=False)
    umap_dists_dss[dataset] = umap_dists
    np.save(os.path.join(output_folder, 'intermediate_files', 'umap_{}_pdists.npy'.format(n_comps)),
                        umap_dists, allow_pickle=False)

In [ ]:
phate_dists_dss = {}
for ind_ds, dataset in enumerate(datasets):
    output_folder = output_folder_dss[dataset]
    phate_projected = phate_projected_dss[dataset]
    phate_proj = phate_projected['all']
    phate_dists = distance.pdist(phate_proj.T, metric='sqeuclidean') / 2
    phate_dists_dss[dataset] = phate_dists
    np.save(os.path.join(output_folder, 'intermediate_files', 'phate_pdists.npy'),
                        phate_dists, allow_pickle=False)

In [ ]:
fig, axs = get_fig_ax(xlabel='PCA dist.', ylabel='Number of cell-cell pairs', title='PCA distances', sharex=True, sharey='col')

# Visualize the true distances
for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax_row, ax_col = info['plt_coords']
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax.hist(pca_dists_dss[dataset], bins=100, color=info['color'], log=True)
plt.tight_layout()

In [ ]:
fig, axs = get_fig_ax(xlabel='UMAP dist.', ylabel='Number of cell-cell pairs', title='UMAP distances', sharex=True, sharey='col')

# Visualize the true distances
for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax_row, ax_col = info['plt_coords']
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax.hist(umap_dists_dss[dataset], bins=100, color=info['color'], log=True)
plt.tight_layout()

In [ ]:
fig, axs = get_fig_ax(xlabel='PHATE dist.', ylabel='Number of cell-cell pairs', title='PHATE distances', sharex=True, sharey='col')

# Visualize the true distances
for ind_ds, dataset in enumerate(datasets):
    info = info_dss[dataset]
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax_row, ax_col = info['plt_coords']
    if len(cell_numbers) == 1:
        ax = axs[ax_row]
    else:
        ax = axs[ax_row, ax_col]
    ax.hist(phate_dists_dss[dataset], bins=100, color=info['color'], log=True)
plt.tight_layout()

## Visualize all PCA and UMAP distances

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=len(datasets), figsize=(10, 4), sharey=True)
dist_objcts = []
for ind_ds, dataset in enumerate(datasets):
    output_folder = output_folder_dss[dataset]
    alldistfiles = list(Path(os.path.join(output_folder, 'intermediate_files')).glob('*_pdists.npy'))
    dataset_objs = []
    for distfile in alldistfiles:
        data_type = os.path.basename(distfile).split("_pdists")[0]
        info = info_dss[dataset]
        num_dims = info['n_gene']
        data_id = data_type + ' {}_dims'.format(num_dims)
        dataset_objs.append(
            Dataset(pdist_file=distfile, data_type=data_type, data_id=data_id, color_types=['sanity', 'bonsai', 'pca', 'umap', 'phate']))
        dataset_objs[-1].n_genes = num_dims
        if data_type[:10] != 'delta_true':
            dist_objcts.append(dataset_objs[-1])
        # if data_type == 'bonsai':
        #     bonsai_dist_objcts.append(datasets[-1])
        # elif data_type[:4] == 'umap':
        #     umap_dist_objcts.append(datasets[-1])
        # elif data_type[:3] == 'pca':
        #     pca_dist_objcts.append(datasets[-1])

    axs_col = axs[ind_ds]
    # avg_rel_diff_list = compare_pdists_to_truth(dataset_subset, make_fig=True, axs=axs_col, axs2=axs2, axs3=axs3,
    #                                             YLABEL=ind_dim == 0,
    #                                             first_title='{}-cells per cluster'.format(n_cells_per_clst))
    avg_rel_diff_list, R_vals = compare_pdists_to_truth_per_cell(dataset_objs, make_fig=True, axs=axs_col,
                                                                          set_lims=False,
                                                                          return_Rvals=True, flip_axes=False,
                                                                          YLABEL=ind_ds == 0, share_y=True,
                                                                          first_title='Number of genes: {}'.format(
                                                                              num_dims), density=True, bins=50,
                                                                          XLABEL=ind_ds == 0, loglog_corr=False)

In [ ]:
pearsonRSqs = []
logRatios = []
# pearsonRSqs_bnsi = []
# pearsonRSqs_umap = []
# pearsonRSqs_pca = []
dataset_names = []
plot_colors = []
ncpcs = []
for ind_dataset, dist_objct in enumerate(dist_objcts):
    data_type = dist_objct.data_type
    if data_type[:4] == 'umap':
        data_type = 'UMAP'
    elif data_type[:3] == 'pca':
        data_type = 'PCA'
    elif data_type[:3] == 'phate':
        data_type = 'PHATE'
    # dataset_names.append("{}\n{}-cell-clusters".format(data_type, dist_objct.n_cells_per_clst))
    dataset_names.append("{}".format(data_type))
    ncpcs.append(dist_objct.n_genes)
    pearsonRSqs.append(dist_objct.pearsonRs ** 2)
    logRatios.append(-np.log10(1-dist_objct.pearsonRs ** 2))
    plot_colors.append(dist_objct.data_type_color)

combined = list(zip(dataset_names, pearsonRSqs, logRatios, plot_colors, ncpcs))
combined_sorted = sorted(combined, key=lambda x: (x[0].lower(), x[4]))
dataset_names_sorted, pearsonRSqs_sorted, logRatios_sorted, plot_colors_sorted, ncpcs_sorted = zip(*combined_sorted)
dataset_names_sorted = [(ds_name.capitalize() if ds_name == 'bonsai' else ds_name) for ds_name in dataset_names_sorted]
n_boxes = len(datasets)

"""Create boxplots for Pearson R-values."""
RSQ = True

boxprops = dict(linewidth=0.05)
flierprops = dict(markersize=2, markeredgewidth=0.5)
medianprops = dict(color='black', linewidth=1)
fig_bp, axs_bp = plt.subplots(figsize=(9, 7), nrows=1, ncols=1)
ax = axs_bp
if RSQ:
    bplot = ax.boxplot(pearsonRSqs_sorted, whis=(5, 95), labels=ncpcs_sorted, patch_artist=True, flierprops=flierprops,
                       medianprops=medianprops, boxprops=boxprops)
else:
    bplot = ax.boxplot(logRatios_sorted, whis=(5, 95), labels=ncpcs_sorted, patch_artist=True, flierprops=flierprops,
                       medianprops=medianprops, boxprops=boxprops)
# fill with colors
for ind_patch, patch in enumerate(bplot['boxes']):
    patch.set_facecolor(color=plot_colors_sorted[ind_patch])
    if ind_patch % n_boxes == int(n_boxes/2):
        ax.text(ind_patch + 1, 0.95, dataset_names_sorted[ind_patch], ha='center', va='bottom', transform=ax.get_xaxis_transform(), fontsize=14)

# ax.set_xticks(xticks, xtick_labels, rotation=45, ha='right')
ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right', fontsize=10)
ax.set_xlabel("Number of dimensions in dataset")

if RSQ:
    ax.set_ylim(-0.05, 1.05)
    # ax.set_ylabel(r'Pearson $R^2$-values between' + "\ntrue and inferred distances\nof each cell to all others")
    ax.set_ylabel(r'Per-cell correlations (Pearson $R^2$)')
else:
    ax.set_ylabel(r'-log10(1-Pearson $R^2$) for $R^2$ between' + "\ntrue and inferred distances\nof each cell to all others")
plt.tight_layout()

In [ ]:
fig_bp.savefig(os.path.join(os.getcwd(), 'boxplots_nonoise_dataset_accuracy_of_distance_estimates.png'), dpi=600)
# fig_bp.savefig(os.path.join(os.getcwd(), 'boxplots_nonoise_dataset_accuracy_of_distance_estimates.svg'))
print("Stored figures at: {}".format(os.path.join(os.getcwd(), 'boxplots_nonoise_dataset_accuracy_of_distance_estimates.png')))